In [1]:
import requests
import urllib.request
from urllib.parse import quote
import numpy as np
from tqdm import tqdm
import contextlib
import json
import tqdm
from bs4 import BeautifulSoup as bs
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import collections
import pandas as pd

### Set up spotify scrapping functions

In [86]:
token = 'BQBxnEgIZeWLUFaY9b_ECTaH31wltOVjaBKKlFy3fzBH3LzuvTFPPfd8maDuwaku_gyurkYJtHgStuip7mmjXwqtAfGu4e7d6OKJ1z_uIkLuZNS2IIaJexS1ZGcOVXiROHteDXUMvbuO2Q'

header = {
    'Authorization': 'Bearer ' + token,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

In [3]:
final_json = []
intermediary_results = []

In [39]:
def song_features(song_id):
    query = 'audio-features/' + str(song_id)
    r = requests.request('get', 'https://api.spotify.com/v1/' + query, headers=header)
    return r.json()


def get_rateYourMusic_songs(genre, filenames, table_index=3) :
    
    #song year, artist and title
    searches = []
    all_info = []
    
    for f in filenames :

        res = bs(open(f),"html5lib")
        for t in res.findAll(attrs={'class': "mbgen"})[table_index].findAll("tr")[1:] :
                        
            if len(t.findAll("td")) >= 3 :
                
                
            
                year = t.findAll("td")[0].text
                artist = t.findAll("td")[1].text
                title = t.findAll("td")[2].text

                if '/' in title :
                    searches.append([year, artist, title.split('/')[0].strip()])
                    searches.append([year, artist, title.split('/')[1].strip()])

                else :
                    searches.append([year, artist, title])

    found = 0

    for e, s in tqdm.tqdm(enumerate(searches)) :
        result  = requests.request('get', 'https://api.spotify.com/v1/search?q='+ quote(s[2]) + "%20artist:" + quote(s[1]) + "&type=track", headers=header).json()
        time.sleep(0.3)
        items = result['tracks']['items']
                        
        
        #We found the song
        if len(items) != 0 and ('error' not in result.keys()) :            
            
            
            song_id = result['tracks']['items'][0]['id']   
            song_release = items[0]['album']['release_date']
            features = song_features(song_id)
            
            if 'error' not in features.keys() :
                
                found += 1
            
                if e >= 90 and e < 110 :
                    print(features)



                #get the preview url
                result_preview  = requests.request('get', "https://api.spotify.com/v1/tracks/" + str(song_id), headers=header).json()
                time.sleep(0.3)
                preview = result_preview['preview_url']

                if preview is None :
                    preview = ""

                year = s[0]
                cover = ""

                if len(items[0]['album']['images']) != 0 :                
                    cover = items[0]['album']['images'][0]['url']            
                else :
                    cover = "https://vignette.wikia.nocookie.net/uncyclopedia/images/a/a0/Questionmark.jpg/revision/latest?cb=20090616065802&format=original"

                artists = s[1]
                name = s[2]
                album = items[0]['album']['name']



                song_dict = {'year' : year,
                             'artists' : artists,
                             'name' : name,
                             'cover' : cover,
                             'album' : album,
                             'genre' : genre,
                            'danceability' : features['danceability'],
                            'energy' : features['energy'],
                            'key' : features['key'],
                            'loudness' : features['loudness'],
                            'mode' : features['mode'],
                            'speechiness' : features['speechiness'],
                            'acousticness' : features['acousticness'],
                            'instrumentalness' : features['instrumentalness'],
                            'liveness' : features['liveness'],
                            'valence' : features['valence'],
                            'tempo' : features['tempo'],
                            'duration_ms' : features['duration_ms'],
                            'preview' : preview}


                all_info.append(song_dict)
                
    print("found : ", found , ", (" + str(100*found/len(searches)) + "%) for genre " + genre)
    return all_info


# Download Contemporary RnB titles (94 songs)

From Itunes (70)

In [96]:
final_json_bis = [f for f in final_json_bis if f['genre'] != 'contemporary-rnb']
len(final_json_bis)

1707

In [97]:
# get names from itunes
url = 'https://itunes.apple.com/search?term=song&genreId=1136&limit=200'

response = requests.get(url)
data = response.json()

found = 0

for e in tqdm.tqdm(data['results']):
    result  = requests.request('get', 'https://api.spotify.com/v1/search?q='+ quote(e['trackName']) + "%20artist:" + quote(e['artistName']) + "&type=track", headers=header).json()
    
    items = result['tracks']['items']
    
    
    #We found the song
    if len(items) != 0 and ('error' not in result.keys()):
        
        found += 1
    
        song_id = result['tracks']['items'][0]['id']    
        features = song_features(song_id)
        
        #get the preview url
        result_preview  = requests.request('get', "https://api.spotify.com/v1/tracks/" + str(song_id), headers=header).json()
        time.sleep(0.3)
        preview = result_preview['preview_url']

        if preview is None :
            preview = ""

        year = e['releaseDate'][:4]
        cover = e['artworkUrl100']
        artists = " and ".join([a['name'] for a in result['tracks']['items'][0]['artists']])
        name = e['trackName']
        album = result['tracks']['items'][0]['album']['name']



        song_dict = {'year' : year,
                     'artists' : artists,
                     'name' : name,
                     'cover' : cover,
                     'album' : album,
                     'genre' : "contemporary-rnb",
                    'danceability' : features['danceability'],
                    'energy' : features['energy'],
                    'key' : features['key'],
                    'loudness' : features['loudness'],
                    'mode' : features['mode'],
                    'speechiness' : features['speechiness'],
                    'acousticness' : features['acousticness'],
                    'instrumentalness' : features['instrumentalness'],
                    'liveness' : features['liveness'],
                    'valence' : features['liveness'],
                    'tempo' : features['tempo'],
                    'duration_ms' : features['duration_ms'],
                    'preview' : preview}


        final_json.append(song_dict)










  0%|          | 0/95 [00:00<?, ?it/s]








  1%|          | 1/95 [00:00<00:54,  1.72it/s]








  2%|▏         | 2/95 [00:01<00:54,  1.70it/s]








  3%|▎         | 3/95 [00:01<00:54,  1.68it/s]








  4%|▍         | 4/95 [00:02<00:53,  1.69it/s]








  5%|▌         | 5/95 [00:02<00:53,  1.69it/s]








  6%|▋         | 6/95 [00:03<00:51,  1.73it/s]








  7%|▋         | 7/95 [00:04<00:51,  1.71it/s]








  8%|▊         | 8/95 [00:04<00:50,  1.73it/s]








  9%|▉         | 9/95 [00:05<00:49,  1.72it/s]








 11%|█         | 10/95 [00:05<00:49,  1.72it/s]








 12%|█▏        | 11/95 [00:06<00:48,  1.72it/s]








 13%|█▎        | 12/95 [00:07<00:48,  1.71it/s]








 14%|█▎        | 13/95 [00:07<00:48,  1.70it/s]








 16%|█▌        | 15/95 [00:08<00:40,  1.95it/s]








 17%|█▋        | 16/95 [00:08<00:42,  1.87it/s]








 18%|█▊        | 17/95 [00:09<00:42,  1.82it/s]








 19%|█▉        | 18/95 [00:10<00:43,  1.78it/s]








 

From rateYourMusic (24 songs)

In [98]:
final_json.extend(get_rateYourMusic_songs("contemporary-rnb", ["data/contemporary_rnb/cont_songs_1.html"], table_index=2))
intermediary_results = final_json.copy()










0it [00:00, ?it/s]








1it [00:00,  1.19it/s]








2it [00:01,  1.16it/s]








3it [00:02,  1.14it/s]








4it [00:03,  1.14it/s]








5it [00:04,  1.13it/s]








6it [00:04,  1.35it/s]








7it [00:05,  1.26it/s]








8it [00:06,  1.22it/s]








9it [00:07,  1.41it/s]








10it [00:08,  1.29it/s]








11it [00:08,  1.23it/s]








12it [00:09,  1.45it/s]








13it [00:09,  1.64it/s]








14it [00:10,  1.45it/s]








15it [00:11,  1.34it/s]








16it [00:12,  1.28it/s]








17it [00:12,  1.46it/s]








18it [00:13,  1.66it/s]








19it [00:14,  1.46it/s]








20it [00:15,  1.34it/s]








21it [00:15,  1.28it/s]








22it [00:16,  1.22it/s]








23it [00:17,  1.42it/s]








24it [00:17,  1.62it/s]








25it [00:18,  1.77it/s]








26it [00:18,  1.94it/s]








27it [00:19,  1.57it/s]








28it [00:19,  1.78it/s]








29it [00:20,  1.93it/s]








30it [00:20,  2.01it/s]








31it [00:21, 

found :  24 , (60.0%) for genre contemporary-rnb


# Download british RnB (527 songs)

In [7]:
final_json.extend(get_rateYourMusic_songs("british-rnb", ["data/british_rnb/brit_songs_" + str(c) + ".html" for c in range(1, 27) ]))
intermediary_results = final_json.copy()

100%|██████████| 856/856 [10:27<00:00,  1.19it/s]

found :  527 , (61.56542056074766%) for genre british-rnb


# Download alternative RnB (198 songs)

In [8]:
final_json.extend(get_rateYourMusic_songs("alternative-rnb", ["data/alternative_rnb/alt_songs_" + str(c) + ".html" for c in range(1, 12) ]))
intermediary_results = final_json.copy()

100%|██████████| 278/278 [03:34<00:00,  1.40it/s]

found :  198 , (71.22302158273381%) for genre alternative-rnb


# Download new jack swing RnB (197 songs)

In [9]:
final_json.extend(get_rateYourMusic_songs("new-jack-swing-rnb", ["data/new_jack_swing_rnb/njs_rnb_" + str(c) + ".html" for c in range(1, 12) ]))
intermediary_results = final_json.copy()

100%|██████████| 300/300 [03:41<00:00,  1.70it/s]

found :  197 , (65.66666666666667%) for genre new-jack-swing-rnb


# Download new Orleans RnB (253 songs)

In [49]:
final_json = intermediary_results.copy()

In [44]:
final_json.extend(get_rateYourMusic_songs("new-orleans-rnb", ["data/new_orleans_rnb/new_orleans_rnb" + str(c) + ".html" for c in range(1, 12) ]))
intermediary_results = final_json.copy()









0it [00:00, ?it/s]







1it [00:00,  2.25it/s]







2it [00:00,  2.29it/s]







3it [00:01,  1.73it/s]







4it [00:02,  1.47it/s]







5it [00:03,  1.34it/s]







6it [00:04,  1.25it/s]







7it [00:05,  1.22it/s]







8it [00:06,  1.17it/s]







9it [00:07,  1.15it/s]







10it [00:08,  1.14it/s]







11it [00:08,  1.36it/s]







12it [00:09,  1.28it/s]







13it [00:10,  1.23it/s]







14it [00:11,  1.20it/s]







15it [00:12,  1.18it/s]







16it [00:12,  1.15it/s]







17it [00:13,  1.12it/s]







18it [00:14,  1.11it/s]







19it [00:15,  1.12it/s]







20it [00:16,  1.12it/s]







21it [00:17,  1.12it/s]







22it [00:17,  1.32it/s]







23it [00:18,  1.53it/s]







24it [00:19,  1.38it/s]







25it [00:20,  1.30it/s]







26it [00:20,  1.51it/s]







27it [00:20,  1.68it/s]







28it [00:21,  1.45it/s]







29it [00:22,  1.33it/s]







30it [00:23,  1.25it/s]







31it [00:24,  1.19it/s]







32it [00:25,  

{'danceability': 0.436, 'energy': 0.397, 'key': 5, 'loudness': -11.856, 'mode': 0, 'speechiness': 0.0288, 'acousticness': 0.982, 'instrumentalness': 0.00129, 'liveness': 0.124, 'valence': 0.467, 'tempo': 92.424, 'type': 'audio_features', 'id': '5QwtuWDP5bywsGVFNd86fX', 'uri': 'spotify:track:5QwtuWDP5bywsGVFNd86fX', 'track_href': 'https://api.spotify.com/v1/tracks/5QwtuWDP5bywsGVFNd86fX', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5QwtuWDP5bywsGVFNd86fX', 'duration_ms': 133947, 'time_signature': 4}










92it [01:14,  1.26it/s]







93it [01:15,  1.48it/s]

{'danceability': 0.739, 'energy': 0.561, 'key': 10, 'loudness': -7.951, 'mode': 1, 'speechiness': 0.0528, 'acousticness': 0.948, 'instrumentalness': 0.00343, 'liveness': 0.0898, 'valence': 0.962, 'tempo': 148.831, 'type': 'audio_features', 'id': '0Ylvo2YRkXtMCBfTnkFQD8', 'uri': 'spotify:track:0Ylvo2YRkXtMCBfTnkFQD8', 'track_href': 'https://api.spotify.com/v1/tracks/0Ylvo2YRkXtMCBfTnkFQD8', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0Ylvo2YRkXtMCBfTnkFQD8', 'duration_ms': 134840, 'time_signature': 4}










94it [01:16,  1.34it/s]

...

{'danceability': 0.853, 'energy': 0.391, 'key': 0, 'loudness': -9.783, 'mode': 0, 'speechiness': 0.0623, 'acousticness': 0.352, 'instrumentalness': 0, 'liveness': 0.169, 'valence': 0.968, 'tempo': 130.967, 'type': 'audio_features', 'id': '4GGpZBuCGxB9Aop0oTL4lH', 'uri': 'spotify:track:4GGpZBuCGxB9Aop0oTL4lH', 'track_href': 'https://api.spotify.com/v1/tracks/4GGpZBuCGxB9Aop0oTL4lH', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4GGpZBuCGxB9Aop0oTL4lH', 'duration_ms': 143973, 'time_signature': 4}










109it [01:27,  1.29it/s]

{'danceability': 0.323, 'energy': 0.106, 'key': 1, 'loudness': -13.141, 'mode': 1, 'speechiness': 0.0401, 'acousticness': 0.82, 'instrumentalness': 0, 'liveness': 0.103, 'valence': 0.505, 'tempo': 198.014, 'type': 'audio_features', 'id': '4kzN8jbroqt9tE9QGKCOgn', 'uri': 'spotify:track:4kzN8jbroqt9tE9QGKCOgn', 'track_href': 'https://api.spotify.com/v1/tracks/4kzN8jbroqt9tE9QGKCOgn', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4kzN8jbroqt9tE9QGKCOgn', 'duration_ms': 160107, 'time_signature': 3}










110it [01:28,  1.22it/s]







111it [01:29,  1.20it/s]







112it [01:29,  1.42it/s]







113it [01:31,  1.07s/it]







114it [01:32,  1.02s/it]







115it [01:33,  1.01it/s]







116it [01:34,  1.02it/s]







117it [01:34,  1.23it/s]







118it [01:35,  1.17it/s]







119it [01:35,  1.38it/s]







120it [01:36,  1.58it/s]







121it [01:37,  1.40it/s]







122it [01:38,  1.30it/s]







123it [01:38,  1.51it/s]







124it [01:39,  1.36it/s]







125it [01:40,  1.27it/s]







126it [01:41,  1.21it/s]







127it [01:42,  1.16it/s]







128it [01:43,  1.15it/s]







129it [01:44,  1.14it/s]







130it [01:44,  1.35it/s]







131it [01:45,  1.27it/s]







132it [01:46,  1.22it/s]







133it [01:47,  1.18it/s]







134it [01:48,  1.15it/s]







135it [01:49,  1.13it/s]







136it [01:49,  1.11it/s]







137it [01:50,  1.11it/s]







138it [01:51,  1.10it/s]







139it [01:52,  1.09it/s]







140it [01:53,  1.09it/s]









found :  251 , (63.70558375634518%) for genre new-orleans-rnb


# Download Jump Blues (261 songs)

In [45]:
final_json.extend(get_rateYourMusic_songs("jump-blues", ["data/jump_blues_rnb/jump_blues_" + str(c) + ".html" for c in range(1, 12) ]))
intermediary_results = final_json.copy()









0it [00:00, ?it/s]







1it [00:00,  1.12it/s]







2it [00:01,  1.12it/s]







3it [00:02,  1.33it/s]







4it [00:02,  1.48it/s]







5it [00:03,  1.68it/s]







6it [00:03,  1.84it/s]







7it [00:04,  1.90it/s]







8it [00:04,  2.04it/s]







9it [00:04,  2.09it/s]







10it [00:05,  2.16it/s]







11it [00:06,  1.70it/s]







12it [00:07,  1.47it/s]







13it [00:08,  1.33it/s]







14it [00:08,  1.27it/s]







15it [00:09,  1.21it/s]







16it [00:10,  1.16it/s]







17it [00:11,  1.35it/s]







18it [00:11,  1.55it/s]







19it [00:12,  1.35it/s]







20it [00:13,  1.27it/s]







21it [00:14,  1.20it/s]







22it [00:15,  1.17it/s]







23it [00:16,  1.15it/s]







24it [00:16,  1.37it/s]







25it [00:17,  1.54it/s]







26it [00:18,  1.37it/s]







27it [00:18,  1.30it/s]







28it [00:19,  1.23it/s]







29it [00:20,  1.17it/s]







30it [00:21,  1.36it/s]







31it [00:22,  1.28it/s]







32it [00:22,  

{'danceability': 0.458, 'energy': 0.354, 'key': 0, 'loudness': -10.773, 'mode': 1, 'speechiness': 0.0464, 'acousticness': 0.956, 'instrumentalness': 0.00487, 'liveness': 0.249, 'valence': 0.594, 'tempo': 148.076, 'type': 'audio_features', 'id': '1Z5Y5iEEPoYFnEssfGzhtD', 'uri': 'spotify:track:1Z5Y5iEEPoYFnEssfGzhtD', 'track_href': 'https://api.spotify.com/v1/tracks/1Z5Y5iEEPoYFnEssfGzhtD', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1Z5Y5iEEPoYFnEssfGzhtD', 'duration_ms': 173267, 'time_signature': 4}










91it [01:07,  1.11it/s]







92it [01:08,  1.30it/s]







93it [01:08,  1.51it/s]

{'danceability': 0.773, 'energy': 0.509, 'key': 9, 'loudness': -11.75, 'mode': 0, 'speechiness': 0.0495, 'acousticness': 0.838, 'instrumentalness': 5.09e-06, 'liveness': 0.265, 'valence': 0.874, 'tempo': 87.95, 'type': 'audio_features', 'id': '7fdJcOzvFWtCCLWmfU41ky', 'uri': 'spotify:track:7fdJcOzvFWtCCLWmfU41ky', 'track_href': 'https://api.spotify.com/v1/tracks/7fdJcOzvFWtCCLWmfU41ky', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7fdJcOzvFWtCCLWmfU41ky', 'duration_ms': 174067, 'time_signature': 4}










94it [01:09,  1.37it/s]

...









109it [01:20,  1.36it/s]

{'danceability': 0.419, 'energy': 0.421, 'key': 5, 'loudness': -12.588, 'mode': 0, 'speechiness': 0.036, 'acousticness': 0.966, 'instrumentalness': 0.937, 'liveness': 0.298, 'valence': 0.676, 'tempo': 146.814, 'type': 'audio_features', 'id': '2TGmJDKy8KF9bwN0EE1van', 'uri': 'spotify:track:2TGmJDKy8KF9bwN0EE1van', 'track_href': 'https://api.spotify.com/v1/tracks/2TGmJDKy8KF9bwN0EE1van', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2TGmJDKy8KF9bwN0EE1van', 'duration_ms': 122747, 'time_signature': 4}










110it [01:21,  1.27it/s]







111it [01:22,  1.46it/s]







112it [01:24,  1.08s/it]







113it [01:25,  1.03s/it]







114it [01:26,  1.00it/s]







115it [01:27,  1.02it/s]







116it [01:27,  1.21it/s]







117it [01:28,  1.19it/s]







118it [01:29,  1.17it/s]







119it [01:30,  1.16it/s]







120it [01:31,  1.15it/s]







121it [01:31,  1.16it/s]







122it [01:32,  1.37it/s]







123it [01:32,  1.57it/s]







124it [01:33,  1.76it/s]







125it [01:34,  1.50it/s]







126it [01:34,  1.36it/s]







127it [01:35,  1.27it/s]







128it [01:36,  1.22it/s]







129it [01:37,  1.16it/s]







130it [01:38,  1.36it/s]







131it [01:38,  1.56it/s]







132it [01:39,  1.40it/s]







133it [01:40,  1.31it/s]







134it [01:41,  1.25it/s]







135it [01:42,  1.19it/s]







136it [01:43,  1.15it/s]







137it [01:43,  1.35it/s]







138it [01:44,  1.27it/s]







139it [01:45,  1.21it/s]







140it [01:46,  1.17it/s]









402it [04:59,  1.57it/s]







403it [05:00,  1.74it/s]







404it [05:01,  1.47it/s]







405it [05:02,  1.26it/s]







406it [05:03,  1.24it/s]







407it [05:03,  1.45it/s]







408it [05:04,  1.64it/s]







409it [05:04,  1.81it/s]







410it [05:04,  1.97it/s]







411it [05:05,  2.08it/s]







412it [05:05,  2.17it/s]







413it [05:06,  1.60it/s]







414it [05:07,  1.77it/s]









found :  259 , (62.56038647342995%) for genre jump-blues


# Download blue-eyed soul (275 songs)

In [50]:
final_json.extend(get_rateYourMusic_songs("blue-eyed-soul", ["data/blue_eyed_soul/blue_eyed_soul_" + str(c) + ".html" for c in range(1, 12) ]))










0it [00:00, ?it/s]








1it [00:00,  1.09it/s]








2it [00:01,  1.11it/s]








3it [00:02,  1.11it/s]








4it [00:03,  1.33it/s]








5it [00:03,  1.27it/s]








6it [00:04,  1.23it/s]








7it [00:05,  1.19it/s]








8it [00:06,  1.17it/s]








9it [00:07,  1.17it/s]








10it [00:08,  1.16it/s]








11it [00:09,  1.15it/s]








12it [00:10,  1.14it/s]








13it [00:11,  1.13it/s]








14it [00:11,  1.11it/s]








15it [00:12,  1.10it/s]








16it [00:13,  1.10it/s]








17it [00:14,  1.30it/s]








18it [00:15,  1.26it/s]








19it [00:15,  1.23it/s]








20it [00:16,  1.19it/s]








21it [00:17,  1.18it/s]








22it [00:18,  1.15it/s]








23it [00:19,  1.14it/s]








24it [00:20,  1.09it/s]








25it [00:21,  1.10it/s]








26it [00:22,  1.09it/s]








27it [00:23,  1.09it/s]








28it [00:24,  1.11it/s]








29it [00:25,  1.13it/s]








30it [00:25,  1.13it/s]








31it [00:26, 

{'danceability': 0.591, 'energy': 0.726, 'key': 2, 'loudness': -13.545, 'mode': 1, 'speechiness': 0.0315, 'acousticness': 0.0384, 'instrumentalness': 2.49e-06, 'liveness': 0.119, 'valence': 0.968, 'tempo': 129.057, 'type': 'audio_features', 'id': '1VZ04sZSpTet7wVMyZphA8', 'uri': 'spotify:track:1VZ04sZSpTet7wVMyZphA8', 'track_href': 'https://api.spotify.com/v1/tracks/1VZ04sZSpTet7wVMyZphA8', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1VZ04sZSpTet7wVMyZphA8', 'duration_ms': 214960, 'time_signature': 4}











91it [01:08,  1.70it/s]

{'danceability': 0.752, 'energy': 0.445, 'key': 4, 'loudness': -12.725, 'mode': 0, 'speechiness': 0.0338, 'acousticness': 0.5, 'instrumentalness': 0, 'liveness': 0.0912, 'valence': 0.964, 'tempo': 106.283, 'type': 'audio_features', 'id': '3MtP9T0WF4xbIo8vqHYeXM', 'uri': 'spotify:track:3MtP9T0WF4xbIo8vqHYeXM', 'track_href': 'https://api.spotify.com/v1/tracks/3MtP9T0WF4xbIo8vqHYeXM', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3MtP9T0WF4xbIo8vqHYeXM', 'duration_ms': 164173, 'time_signature': 4}











92it [01:09,  1.48it/s]

...










107it [01:21,  1.37it/s]

{'danceability': 0.65, 'energy': 0.69, 'key': 11, 'loudness': -8.433, 'mode': 0, 'speechiness': 0.104, 'acousticness': 0.0765, 'instrumentalness': 0.00104, 'liveness': 0.0699, 'valence': 0.797, 'tempo': 122.311, 'type': 'audio_features', 'id': '642VUaztZTooWDBDP98Vtz', 'uri': 'spotify:track:642VUaztZTooWDBDP98Vtz', 'track_href': 'https://api.spotify.com/v1/tracks/642VUaztZTooWDBDP98Vtz', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/642VUaztZTooWDBDP98Vtz', 'duration_ms': 180800, 'time_signature': 4}











108it [01:22,  1.28it/s]








109it [01:23,  1.49it/s]








110it [01:23,  1.67it/s]








111it [01:23,  1.85it/s]








112it [01:24,  1.58it/s]








113it [01:25,  1.41it/s]








114it [01:26,  1.57it/s]








115it [01:26,  1.78it/s]








116it [01:27,  1.51it/s]








117it [01:27,  1.71it/s]








118it [01:28,  1.88it/s]








119it [01:29,  1.56it/s]








120it [01:30,  1.40it/s]








121it [01:30,  1.62it/s]








122it [01:30,  1.79it/s]








123it [01:31,  1.52it/s]








124it [01:32,  1.72it/s]








125it [01:33,  1.49it/s]








126it [01:33,  1.69it/s]








127it [01:33,  1.89it/s]








128it [01:34,  1.54it/s]








129it [01:35,  1.39it/s]








130it [01:36,  1.32it/s]








131it [01:37,  1.26it/s]








132it [01:38,  1.20it/s]








133it [01:39,  1.17it/s]








134it [01:40,  1.17it/s]








135it [01:40,  1.39it/s]








136it [01:40,  1.58it/s]








137it [01:41,  1.42it/s]








1

380it [04:51,  1.30it/s]








381it [04:52,  1.24it/s]








382it [04:52,  1.46it/s]








383it [04:53,  1.33it/s]








384it [04:54,  1.26it/s]








385it [04:55,  1.18it/s]








386it [04:56,  1.14it/s]










found :  275 , (71.24352331606218%) for genre blue-eyed-soul


In [51]:
#Dump everything into the data
json.dump(final_json, open("rnb_data.json", 'w'))

# visualize number of songs per year

In [ ]:
import collections

#all possible genres
genres = ['contemporary-rnb', 'british-rnb', 'new-orleans-rnb', 
          'alternative-rnb', 'new-jack-swing-rnb', 'blue-eyed-soul', 'jump-blues']

#compute earliest and latest year globally
all_years = [d['year'] for d in final_json if (d['year'] != '')]
min_all_years, max_all_years = int(min(all_years)), int(max(all_years)) + 1

#plot settings
width = 1
figure(figsize=(15,10))
plots = []

#initialize all years to zero
seen_years = collections.Counter()
for y in range(min_all_years, max_all_years) :
        seen_years[str(y)] = 0



for g in genres :

    #years of genre songs
    genre_years = [d['year'] for d in final_json if ((d["genre"] == g) and (d['year'] != ''))]
    
    #create counter
    years_per_genre = collections.Counter()
    
    
    #put 0 as default value for all years
    for y in range(min_all_years, max_all_years) :
        years_per_genre[str(y)] = 0
        
    #count values
    years_per_genre.update(genre_years)
    
    #create plot variables
    labels, values = zip(*years_per_genre.items())
    labels_all, values_all = zip(*seen_years.items())
    indexes = np.arange(len(labels))    
    
    #plot with bottom
    p = plt.bar(indexes, values, width, bottom=values_all)    
    plots.append(p)
    
    #update seen years counter
    seen_years.update(genre_years)

#settings for axes
plt.xticks(indexes + width * 0.5, labels, rotation=90)
plt.xlabel("release year")
plt.ylabel("number of songs")
plt.ylim(ymax = 160, ymin = 0)

#general plot settings
plt.title("number of RnB songs per genre and per year")
plt.legend(plots, genres)

# plot
plt.savefig("nb_of_songs.png")
plt.show()


# compute discriminative coefficients for each feature and each year 

In [94]:
[f for f in final_json_bis if f['genre'] == 'contemporary-rnb']

[{'year': '1979',
  'artists': "The O'Jays",
  'name': 'Forever Mine',
  'cover': 'https://is4-ssl.mzstatic.com/image/thumb/Music/v4/dd/aa/c7/ddaac7ba-436e-9bbc-5549-be612af19b1c/source/100x100bb.jpg',
  'album': 'Identify Yourself',
  'genre': 'contemporary-rnb',
  'danceability': 0.483,
  'energy': 0.416,
  'key': 7,
  'loudness': -11.85,
  'mode': 0,
  'speechiness': 0.0588,
  'acousticness': 0.288,
  'instrumentalness': 0,
  'liveness': 0.145,
  'valence': 0.145,
  'tempo': 178.175,
  'duration_ms': 366227,
  'preview': ''},
 {'year': '2014',
  'artists': 'Ashanti and French Montana',
  'name': 'Early In the Morning (feat. French Montana)',
  'cover': 'https://is5-ssl.mzstatic.com/image/thumb/Music6/v4/f7/e1/27/f7e12715-eae4-f03e-ecc7-a01c40b6d0b3/source/100x100bb.jpg',
  'album': 'Braveheart',
  'genre': 'contemporary-rnb',
  'danceability': 0.685,
  'energy': 0.647,
  'key': 10,
  'loudness': -8.784,
  'mode': 1,
  'speechiness': 0.0357,
  'acousticness': 0.174,
  'instrumentalne

In [92]:
years = range(1900, 2050)
best_features = dict()

for y in years :
    
    # load data
    names = ['energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    targets = np.array([c['genre'] for c in final_json if c['year'] == str(y)])
    
    if len(targets) != 0 and  : 
        print("passed")

        features = np.array([c['danceability'] for c in final_json if c['year'] == str(y)])
        
        
        for n in names :
            features = np.vstack((features, np.array([c[n] for c in final_json if c['year'] == str(y)])))


        model = LogisticRegression()
        rfe = RFE(model, 2)
        fit = rfe.fit(features.T, targets)

        best_for_year = [c[0] for c in zip(['danceability'] + names, fit.support_) if c[1]]
        best_features[y] = best_for_year
        
best_features

passed


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 'jump-blues'